# Generating dataset

In [1]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")


# The OS module in Python provides functions for interacting with the operating system.
import os

# Matplotlib is a data visualization and graphical plotting library for Python.
import matplotlib.pyplot as plt

# seaborn is alse a data visualization and graphical plotting library for Python.
import seaborn as sn

# used to display markdown,image,control (frontend utilities)
from IPython.display import display, clear_output

# computer vision library
import cv2

import sys

import mediapipe as mp

import time

import random

In [2]:
PATH=r"D:/Image_datasets/asl_dataset"

In [3]:
# view directory tree
def tree_printer(root):
    try:
        if ":" not in root:
            root=os.getcwd().replace("\\","/")+"/"+root
            print(root)
        if not os.path.isdir(root) and root!="":
            print(root,": path not exists....")
            return
    except:
        print("🛠️ set path of the data set from your local mechine")
        return
    
    
    print("🗁",root,"-->",len(os.listdir(os.path.join(root))), "Items present.")
    for name in os.listdir(root):
        try:
            print(" |- 🗁",name," "*2+"🏴",len(os.listdir(os.path.join(root, name))),"items")
        except:
            continue
            
tree_printer(PATH)

🗁 D:/Image_datasets/asl_dataset --> 36 Items present.
 |- 🗁 0   🏴 70 items
 |- 🗁 1   🏴 70 items
 |- 🗁 2   🏴 70 items
 |- 🗁 3   🏴 70 items
 |- 🗁 4   🏴 70 items
 |- 🗁 5   🏴 70 items
 |- 🗁 6   🏴 70 items
 |- 🗁 7   🏴 70 items
 |- 🗁 8   🏴 70 items
 |- 🗁 9   🏴 70 items
 |- 🗁 a   🏴 70 items
 |- 🗁 b   🏴 70 items
 |- 🗁 c   🏴 70 items
 |- 🗁 d   🏴 70 items
 |- 🗁 e   🏴 70 items
 |- 🗁 f   🏴 70 items
 |- 🗁 g   🏴 70 items
 |- 🗁 h   🏴 70 items
 |- 🗁 i   🏴 70 items
 |- 🗁 j   🏴 70 items
 |- 🗁 k   🏴 70 items
 |- 🗁 l   🏴 70 items
 |- 🗁 m   🏴 70 items
 |- 🗁 n   🏴 70 items
 |- 🗁 o   🏴 70 items
 |- 🗁 p   🏴 70 items
 |- 🗁 q   🏴 70 items
 |- 🗁 r   🏴 70 items
 |- 🗁 s   🏴 70 items
 |- 🗁 t   🏴 65 items
 |- 🗁 u   🏴 70 items
 |- 🗁 v   🏴 70 items
 |- 🗁 w   🏴 70 items
 |- 🗁 x   🏴 70 items
 |- 🗁 y   🏴 70 items
 |- 🗁 z   🏴 70 items


In [4]:
class handDetector:
    def __init__(self, staticImageMode=False, maxNumHands=2, minDetectionConfidence=0.5, trackCon=0.5):
        self.results = None
        self.staticImageMode = staticImageMode
        self.maxNumberHands = maxNumHands
        self.minDetectionConfidence = minDetectionConfidence
        self.trackCon = trackCon
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            static_image_mode=self.staticImageMode,
            max_num_hands=self.maxNumberHands,
            min_detection_confidence=self.minDetectionConfidence,
            min_tracking_confidence=self.trackCon)

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
        # for rect in self.results.hand_rects:
        #     print(rect)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mp_drawing.draw_landmarks(img, handLms,
                                                   self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, maxHandNo=1, draw=False):
        mainlmlist = []
        handsType = []
        # handtype=[0,0]
        if self.results.multi_handedness:
            for hand in self.results.multi_handedness:
                # print(hand)
                # print(hand.classification)
                # print(hand.classification[0])
                handType = hand.classification[0].label
#                 print(handType)
                handsType.append(handType)

        # print(len(self.results.multi_hand_landmarks[0]))
        if self.results.multi_hand_landmarks:
            for myHand in self.results.multi_hand_landmarks:
                lmList = []
                if self.results.multi_hand_landmarks:

                    for pid, lm in enumerate(myHand.landmark):
                        # print(id, lm)
                        h, w, c = img.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        # print(id, cx, cy)
                        lmList.append([pid, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                mainlmlist.append(lmList)

        return mainlmlist, handsType

In [5]:

# load the images from the local mechine and process before putting the CSV.
totalImages=2515


def load_images(path,type=["jpg","jpeg","png"]):
    handDetectorModel=handDetector()
    fileCount=0
    skippedImage=0
    print(path)
    filew = open("dataset.csv", "w")  # write mode
    filew.write("")
    filew.close()
    filea = open("dataset.csv", "a")  # append mode
    
    start_time = time.time()
    
    for  labeldir in os.listdir(path):
        for filepath in os.listdir(os.path.join(path,labeldir)):
            if filepath.split(".")[-1].lower() in type:
                relativePath=labeldir+"/"+filepath
                visualize(fileCount,totalImages,relativePath,int(time.time()-start_time),skippedImage)
                fileCount+=1
                handType,pointslist=trackHand(handDetectorModel,path, relativePath)
                if len(pointslist)==0:
                    skippedImage+=1
                    continue
                writeInCSV(filea,relativePath,handType,labeldir,pointslist)
    visualize(fileCount,totalImages,relativePath,int(time.time()-start_time),skippedImage,last=True) 
                
def trackHand(handDetectorModel,DPath,relativePath):
    frame=cv2.imread(os.path.join(DPath,relativePath))
    frame=cv2.resize(frame, (400, 400))
    clone = handDetectorModel.findHands(frame.copy())
    mainlmList, handsType = handDetectorModel.findPosition(clone, draw=False)
    flattenedList=[]
#     print(mainlmList)
    if len(mainlmList)==0:
        return None,[]
    for keypoint in mainlmList[0]:
        flattenedList.append(keypoint[1])
        flattenedList.append(keypoint[2])
    return handsType[0],flattenedList

def writeInCSV(filea,filename,handtype,label,coordinates):
    points=",".join(list(map(str, coordinates)))
    row="{},{},{},{}".format(filename,handtype,points,label)
    filea.write("{}\n".format(row))

def visualize(count,total,filename,execTime,skipped,last=False):
    if last:
        filename="Completed ...."
    filename=filename+(40-len(filename))*" "
    clear_output(wait=True)
    prog=(40*count)//total
    percent=(count*100)//total
    line="{}/{} [{}{}] {}%".format(count,total,"#"*prog,"-"*(40-prog),percent)
    sys.stdout.write("\rAnalyzing : \033[93m{}\033[0m   | Running for \033[35m{}s\n\033[32m{} | \033[31mskipped {}\033[0m".format(filename, execTime, line, skipped))
    

In [6]:
load_images(PATH)

Analyzing : Completed ....                             | Running for 166s
2515/2515 [########################################] 100% | skipped 0